## Import Libraries

In [ ]:
%store -r
import sys,json, os
import kfp
import kfp.dsl as dsl
from kfp import components
import kfp.compiler as compiler
import kfp.dsl as dsl
from kubernetes import client as k8s_client

import string
import random
import time

from dkube.sdk import *
from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import DkubeModelmonitor
from dkube.sdk.rsrcs.operator import DkubeCluster
from dkube.sdk.rsrcs.modelmonitor import DatasetClass,ModelType,DriftAlgo
from dkube.sdk.rsrcs.modelmonitor import DatasetFormat,DkubeModelmonitorAlert, TimeZone
from dkube.sdk.rsrcs.modelmonitor import DataType, ChannelOrder, ImageDataSavedFileFormat

job_class = os.getenv("DKUBE_JOB_CLASS")
if not job_class:
    !{sys.executable} -m pip install kfp==1.4.0 kfp-server-api==1.2.0 --user >/dev/null

# Set up font definitions for output
class style:
   RED = '\033[91m\033[1m'
   BOLD = '\033[1m'
   END = '\033[0m'

## Setup Configuration Variables from Resources Script

In [ ]:
MONITOR_NAME = image_exp_config['MONITOR_NAME']
INPUT_TRAIN_TYPE = image_exp_config['INPUT_TRAIN_TYPE']
SERVING_DKUBE_USERNAME = image_exp_config['SERVING_DKUBE_USERNAME']
TRAINING_DATASET = image_exp_config['TRAINING_DATASET']
DKUBE_TRAINING_CODE_NAME = image_exp_config['DKUBE_TRAINING_CODE_NAME']
SERVING_DKUBE_URL = image_exp_config['SERVING_DKUBE_URL']
SERVING_DKUBE_TOKEN = image_exp_config['SERVING_DKUBE_TOKEN']

## Define Functions

In [ ]:
def wait_for_deployment_running(deployment_id):
    status = None
    inference_url, inference = None, None
    while True:
        data = serving_api.get_deployment(deployment_id)
        status = data.data.inferenceservice_deployment.parameters.generated.status.state
        inference = data.data.inferenceservice_deployment.parameters.inference
        inference_url = data.data.inferenceservice_deployment.parameters.generated.details.serving.servingurl
        if status == "RUNNING":
            break
        print("waiting for deployment to be running")
        time.sleep(serving_api.wait_interval)
    return inference, inference_url

## Load Components

In [ ]:
dkube_preprocessing_op = components.load_component_from_file("/mnt/dkube/pipeline/components/preprocess/component.yaml")
dkube_training_op = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op  = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml") 

In [ ]:
api = DkubeApi(URL=SERVING_DKUBE_URL, token=SERVING_DKUBE_TOKEN)
client = kfp.Client(
    host=os.getenv("KF_PIPELINES_ENDPOINT"),
    existing_token=SERVING_DKUBE_TOKEN,
    namespace=SERVING_DKUBE_USERNAME)

## Pipeline Inputs

In [ ]:
input_training_dataset = TRAINING_DATASET
training_program = DKUBE_TRAINING_CODE_NAME

## Training stage inputs
input_dataset_mount = ['/data']
training_script = "python image_cloudevents/training.py"
model_name = MONITOR_NAME
output_model_mount = "/model"

## Pipeline Definition

In [ ]:
@kfp.dsl.pipeline(
    name='xray-pipeline',
    description='xray-training-pl'
)
def xray_pipeline(token):    
    train       = dkube_training_op(container=json.dumps({"image": "ocdr/dkube-datascience-tf-cpu:v2.0.0-17"}),
                                    framework="tensorflow", version="2.0.0",
                                    program=str(training_program), 
                                    run_script=str(training_script),
                                    datasets=json.dumps([str(input_training_dataset)]), 
                                    outputs=json.dumps([str(model_name)]),
                                    input_dataset_mounts=json.dumps(input_dataset_mount),
                                    output_mounts=json.dumps([str(output_model_mount)]),
                                    auth_token=token)
    
    serving     = dkube_serving_op(model=train.outputs['artifact'], device='cpu',
                                    name=MONITOR_NAME,
                                    serving_image=json.dumps({"image": "ocdr/tensorflowserver:2.0.0"}),
                                    auth_token=token, min_replicas = '1',
                                    production="true").after(train)

## Create the Pipeline Run to Deploy Model

In [ ]:
# Check if the deployed model already exists
# If not, create the pipeline run to train and deploy the model
serving_api = DkubeApi(URL=SERVING_DKUBE_URL,token=SERVING_DKUBE_TOKEN)
SERVING_DEPLOYMENT_ID = serving_api.get_deployment_id(name=MONITOR_NAME)

# Create pipeline run name
res = ''.join(random.choices(string.ascii_lowercase + string.digits, k=4))
run_name = f"{SERVING_DKUBE_USERNAME}-chest-xray-%s"%res

# Create and run the pipeline
if not SERVING_DEPLOYMENT_ID:
    client.create_run_from_pipeline_func(xray_pipeline, run_name=run_name, arguments={'token':SERVING_DKUBE_TOKEN})
else:
    print(f"{style.BOLD}Deployment already exists, skipping training and deployment{style.END}")

# Wait for serving deployment to come up
while True:
    SERVING_DEPLOYMENT_ID = serving_api.get_deployment_id(name=MONITOR_NAME)
    if SERVING_DEPLOYMENT_ID:
        break
    print("waiting for deployment to come up")
    time.sleep(serving_api.wait_interval)
inference, INFERENCE_URL = wait_for_deployment_running(SERVING_DEPLOYMENT_ID)

# Enable inference logs
if not inference.enable_logs:
    print("Enabling logs")
    serving = DkubeServing(user=SERVING_DKUBE_USERNAME, name=MONITOR_NAME)
    serving.update_enable_logs(enable_logs=True)
    serving_api.update_inference(serving)

# Save deployment url & ID
print("Inference is up at URL: ", INFERENCE_URL)
image_exp_config['INFERENCE_URL'] = INFERENCE_URL
image_exp_config['SERVING_DEPLOYMENT_ID'] = SERVING_DEPLOYMENT_ID
%store image_exp_config

## Fields used for Configuring the Monitor through the UI

In [ ]:
# Print the instructions for the required fields in the UI-Based setup
print()
print(f"{style.BOLD}Note: The Deployment ID will be needed as an input during the UI-Based configuration process{style.END}")
print(f"{style.RED}Deployment ID (for Prefix Fields) = {style.END}", SERVING_DEPLOYMENT_ID)